In [1]:
import pandas as pd
import ast
import fun
import re

In [2]:
# abrir el csv en un DataFrame
df_reviews=pd.read_csv(r'./user_reviews.csv ')

In [3]:
df_reviews.head(3)

,user_id,user_url,item_id,funny,posted,last_edited,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,NaN,"Posted November 5, 2011.",NaN,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,NaN,"Posted July 15, 2011.",NaN,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,NaN,"Posted April 21, 2011.",NaN,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


- Dado que se debe crear una columna  'sentiment_analysis' usando las reseñas de juegos hechas por los usuarios es nesario depurar algo de informacion de esta columna 'reviews ', por ende se explicara a groso modo cada columna : 

    - **user_id&emsp;:**&emsp;Identificador único para el usuario.
    - **user_url&emsp;:**&emsp;Direccion url del perfil del usuario en streamcommunity.
    - **item_id&emsp;:**&emsp;Identificador único del item que seria el id del juego.
    - **funny&emsp;:**&emsp;Número de personas que marcaron esto comenta que es gracioso.
    - **posted&emsp;:**&emsp;Fecha de posteo del review.
    - **last_edited&emsp;:**&emsp;Fecha de la última edición del review.
    - **helpful:&emsp;:**&emsp;Número de personas que marcaron el review ('reseña') como útil.
    - **recommend&emsp;:**&emsp;Recomendado por el usuario (Si o No = True o False).
    - **review&emsp;:**&emsp;Texto con la reseña sobre un juego escrita por un usuario. 
</br>
</br>
- Las columnas 'reviews_funny', 'reviews_last_edited' y 'helpful' son columnas innecesarias puesto que 'funny' aplica solo si el comentario fue grasioso para otros usuarios, 'last_edited' es la fecha de la ultima edicion de la reseña y 'helpful' valora si la reseña de un usuario fue de ayuda para otros usuarios pero existen muchas filas que tienen valores como 'No ratings yet', es decir sin valoracion por cual no es necesria dicha columna.
- La columna 'user_url' es innecesaria, se eliminara

- En la columna 'posted' ahi que quitarle la palabra 'Posted' que aparece antes de la fecha, de igual manera las fechas es mejor manejalos con  formato YYYY-MM-DD

In [4]:
# Eliminacion de columnas
df_reviews = df_reviews.drop(columns=['funny','last_edited','helpful','user_url'])
df_reviews.columns

Index(['user_id', 'item_id', 'posted', 'recommend', 'review'], dtype='object')

In [5]:
# Se verifica si existen valores nulos y el tipo de dato de cada columna
fun.tipo_dato(df_reviews)

user_id (nulos) =  0
item_id (nulos) =  0
posted (nulos) =  0
recommend (nulos) =  0
review (nulos) =  30

 filas completamente nulas:  0


,Nombre_Columna,Tipo_dato,%_No_nulos,%_Nulos
0,user_id,[<class 'str'>],100.00,0.00
1,item_id,[<class 'int'>],100.00,0.00
2,posted,[<class 'str'>],100.00,0.00
3,recommend,[<class 'bool'>],100.00,0.00
4,review,"[<class 'str'>, <class 'float'>]",99.95,0.05


In [6]:
# Se revisan si existen filas duplicadas en el DataFrame 
duplicados =  df_reviews.loc[df_reviews.duplicated()]
duplicados

,user_id,item_id,posted,recommend,review
1112,bokkkbokkk,346110,"Posted September 24, 2015.",True,yep
2891,ImSeriouss,218620,"Posted January 10, 2014.",True,"Good graphics, fun heists! A bit laggy"
2892,ImSeriouss,105600,"Posted January 10, 2014.",True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2893,ImSeriouss,570,"Posted December 17, 2014.",True,bobo pinoy
2894,ImSeriouss,211820,"Posted January 13, 2014.",True,If you want to play this game.. expect glithes...
...,...,...,...,...,...
44433,76561198092022514,422400,Posted July 3.,True,Muy entretenido y una coleccion de armas prome...
44434,76561198092022514,218620,Posted June 1.,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44435,76561198092022514,261820,"Posted August 17, 2014.",True,"Buen juego, no importa el desarrrollo que tien..."
44436,76561198092022514,224260,"Posted February 17, 2014.",True,exelente aporte :D¬°¬°¬° es una buen mod basad...


In [7]:
# se eliminan los 874 datos duplicados
df_reviews1 = df_reviews.drop_duplicates(keep='first') # se conserva la primera aparición de esa fila y se eliminan las apariciones duplicadas subsiguientes.

### Columna 'item_id'

In [8]:
# Se renombra la columna 'item_id'
df_reviews = df_reviews.rename(columns={'item_id': 'id_game'})
df_reviews['id_game'] = df_reviews['id_game'].astype(int)
df_reviews.columns

Index(['user_id', 'id_game', 'posted', 'recommend', 'review'], dtype='object')

### Columna 'posted'

In [9]:
df_reviews['posted'].unique()

array(['Posted November 5, 2011.', 'Posted July 15, 2011.',
       'Posted April 21, 2011.', ..., 'Posted February 18, 2013.',
       'Posted November 13, 2012.', 'Posted November 3, 2012.'],
      dtype=object)

Se observa la palabra 'posted' al inicio de cada fecha, se procede a cambiar la fecha a un formato 'yyyy-mm-dd', de igual manera solo se necesita solamente el año y existen datos que no tienen por ende se eliminaran dado que no se podria generar una fecha exacta para los susuarios que genraron dicho post entonces se opta por la eliminacion

In [10]:
# Funcion para convertir fecha

def convertir_fecha(cadena_fecha):
#\w+: es una clase de caracteres que representa caracteres alfabéticos (letras, números o guiones bajos), y el + significa una o más ocurrencias.
#\s: Esto coincide con un espacio en blanco.
#\d{1,2}: \d representa cualquier dígito (0-9), y {1,2} significa que esperamos uno o dos dígitos.

    match = re.search(r'(\w+\s\d{1,2},\s\d{4})', cadena_fecha) # Busca un patrón en la cadena de fecha que coincida con el formato "Mes Día, Año".
    if match:
        fecha_str = match.group(1) # Extrae el texto coincidente, que es la fecha en formato "Mes Día, Año".
        try:
            fecha_dt = pd.to_datetime(fecha_str) # Intenta convertir la fecha en un objeto datetime de Pandas.
            return fecha_dt.strftime('%Y-%m-%d')# Formatea la fecha en el formato 'YYYY-MM-DD' y devuelve el resultado.
        except:
            return 'Fecha inválida' # Si hay un error al convertir la fecha, se devuelve 'Fecha inválida'.
    else:
        return 'Formato inválido'# Si no se encuentra un patrón coincidente en la cadena, se devuelve 'Formato inválido'.

In [11]:
# Se convierte la fecha al formato deseado y se crea en una columna llamada 'review_date'
df_reviews['posted_date'] =df_reviews['posted'].apply(convertir_fecha) 
df_reviews.head(2)

,user_id,id_game,posted,recommend,review,posted_date
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,2011-07-15


In [12]:
# Se elimina la columna 'posted'
df_reviews = df_reviews.drop(columns=['posted'])
df_reviews.columns

Index(['user_id', 'id_game', 'recommend', 'review', 'posted_date'], dtype='object')

In [13]:
# Cilco for para poder ver todos los valores de la columna en un texto
especif = df_reviews['posted_date'].unique()
for i in especif:
    print(i)

2011-11-05
2011-07-15
2011-04-21
2014-06-24
2013-09-08
2013-11-29
Formato inválido
2015-12-04
2014-11-03
2014-10-15
2013-10-14
2012-07-28
2012-06-02
2014-06-29
2012-11-22
2012-02-23
2014-04-15
2013-12-23
2014-03-14
2013-07-11
2014-05-05
2012-12-24
2012-10-21
2012-03-20
2012-03-09
2015-02-01
2014-06-20
2014-08-25
2013-12-25
2012-06-23
2015-09-05
2015-03-30
2014-02-19
2014-07-14
2013-04-27
2015-07-20
2013-11-04
2013-07-12
2012-08-19
2015-06-19
2014-09-20
2014-09-07
2014-12-19
2015-02-17
2014-06-07
2014-02-12
2014-02-09
2015-10-31
2015-02-04
2014-04-05
2015-01-08
2015-10-26
2013-03-20
2015-12-28
2015-05-19
2015-12-15
2015-08-21
2014-10-17
2014-06-27
2014-04-28
2015-11-12
2014-06-18
2014-11-20
2014-05-29
2014-12-29
2015-02-13
2014-12-26
2014-02-04
2015-05-21
2013-12-06
2014-05-12
2015-08-24
2015-01-06
2015-12-22
2015-06-23
2015-09-11
2013-04-22
2012-05-27
2015-12-05
2015-10-04
2015-10-03
2015-08-13
2015-02-22
2015-01-14
2013-12-14
2013-12-01
2012-02-11
2015-03-12
2015-10-14
2013-10-27
2015

In [14]:
no_validos = df_reviews[(df_reviews['posted_date']=='Fecha inválida')|(df_reviews['posted_date']=='Formato inválido')]
no_validos

,user_id,id_game,recommend,review,posted_date
6,evcentric,248820,True,A suitably punishing roguelike platformer. Wi...,Formato inválido
27,76561198079601835,730,True,ZIKA DO BAILE,Formato inválido
28,MeaTCompany,730,True,BEST GAME IN THE BLOODY WORLD,Formato inválido
31,76561198156664158,252950,True,love it,Formato inválido
32,76561198077246154,440,True,mt bom,Formato inválido
...,...,...,...,...,...
59300,76561198312638244,70,True,a must have classic from steam definitely wort...,Formato inválido
59301,76561198312638244,362890,True,this game is a perfect remake of the original ...,Formato inválido
59302,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Formato inválido
59303,LydiaMorley,730,True,:D,Formato inválido


In [15]:
# Se eliminan los valores qque tengan 'Formato invalido'
df_reviews = df_reviews[df_reviews['posted_date'].str.strip() != 'Formato inválido']

### Columna 'recommend'

In [16]:
# Pasar a valores booleanos la columna 'recommend'
df_reviews.recommend = df_reviews.recommend.astype(int)
df_reviews.head(4)

,user_id,id_game,recommend,review,posted_date
0,76561197970982479,1250,1,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,1,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,1,I know what you think when you see this title ...,2014-06-24


### Columna 'review	'

In [17]:
# se pasan a str todos los valores de columna review
df_reviews['review'] = df_reviews['review'].astype(str)

In [18]:
# se ordenan las columnas a una manera mas legible
orden=['id_game','review','posted_date','recommend',]
df_reviews = df_reviews[orden]
df_reviews.columns

Index(['id_game', 'review', 'posted_date', 'recommend'], dtype='object')

In [19]:
# Ordenamos los id's game en forma ascendente y hacemos un reset al index 
df_reviews=df_reviews.sort_values(by='id_game', ascending=True)
df_reviews.reset_index(drop=True, inplace=True)
df_reviews

,id_game,review,posted_date,recommend
0,10,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZ√ÅMIT !...,2012-07-24,1
1,10,Good Game :D,2012-04-07,1
2,10,"1,6 > go fk go.",2014-03-23,1
3,10,"Pure, hardcore Counter-Strike in its rawest fo...",2015-10-31,1
4,10,THE BEST FPS GAME!!!!!,2014-01-22,1
...,...,...,...,...
49181,423880,"This game, Is ‚ô•‚ô•‚ô•‚ô•ing glorious.Achieve...",2015-12-12,1
49182,423880,10/10 would waste my time again on games like ...,2015-12-23,1
49183,423880,10/10,2015-12-21,1
49184,423880,Do you like wasting a few minutes of your life...,2015-12-11,1


In [20]:
# Se verifica que no se tenga valores vacios en la columna reviews
if df_reviews['review'].isnull().any():
    print("Hay valores nulos en la columna 'review'.")
else:
    print("No hay valores nulos en la columna 'review'.")

No hay valores nulos en la columna 'review'.


In [21]:
# Capitalizar la primera letra de cada palabra en el nombre de las columnas
def capitalizar(texto):
    return texto.capitalize()
df_reviews.columns = df_reviews.columns.map(capitalizar)
df_reviews.head(5)

,Id_game,Review,Posted_date,Recommend
0,10,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZ√ÅMIT !...,2012-07-24,1
1,10,Good Game :D,2012-04-07,1
2,10,"1,6 > go fk go.",2014-03-23,1
3,10,"Pure, hardcore Counter-Strike in its rawest fo...",2015-10-31,1
4,10,THE BEST FPS GAME!!!!!,2014-01-22,1


In [22]:
df_reviews.to_csv('reviews.csv', index=False)

## Creacion de la columna 'sentiment_analysis'

Para el análisis de sentimiento se utilizara TextBlob, se calculara la polaridad de sentimiento y se clasificara de la siguiente manera  positivo, negativo o neutral, Textblob tiene un rango de -1 a 1 en sentimiento de polaridad, donde valores cercanos a 1 indican un sentimiento positivo y valores cercanos a -1 indican un sentimiento negativo; para este ejercicio las polaridades tendran un rago entre -0.2 y 0.2,es decir polaridades por debajao de -0.2 seran(negativas), por encima de 0.2 seran (positivas), y las esten en medio del rango seran (neutrales).

In [23]:
import nltk # Biblioteca Natural Language Toolkit
from nltk.corpus import stopwords #  Lista de palabras comunes en varios idiomas que se consideran "palabras vacías" 
from textblob import TextBlob # Biblioteca de procesamiento de lenguaje natural (NLP) e
from textblob import Word # Operaciones de procesamiento de texto en palabras individuales.
nltk.download('wordnet') # Base de datos léxica y un recurso muy útil para el procesamiento de lenguaje natural (NLP).
nltk.download('stopwords') # Conjunto de palabras vacías en el idioma inglés.

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Steven\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Steven\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
df_reviews1=df_reviews

In [26]:
df_reviews1['Review'] = df_reviews1['Review'].astype(str)

In [27]:
# se paasa a minúsculas todas las palabras de la columna
df_reviews1['Review'] = df_reviews1['Review'].str.lower()

In [28]:
df_reviews1

,Id_game,Review,Posted_date,Recommend
0,10,gyertek gamelni mindenki itt van aki sz√åmit !...,2012-07-24,1
1,10,good game :d,2012-04-07,1
2,10,"1,6 > go fk go.",2014-03-23,1
3,10,"pure, hardcore counter-strike in its rawest fo...",2015-10-31,1
4,10,the best fps game!!!!!,2014-01-22,1
...,...,...,...,...
49181,423880,"this game, is ‚ô•‚ô•‚ô•‚ô•ing glorious.achieve...",2015-12-12,1
49182,423880,10/10 would waste my time again on games like ...,2015-12-23,1
49183,423880,10/10,2015-12-21,1
49184,423880,do you like wasting a few minutes of your life...,2015-12-11,1


In [29]:
# Utiliza una expresión regular para eliminar caracteres especiales y mantener solo letras y espacio, con esto tambien se eliminaran las filas
# que estan en otro idioma y solo dejar las de habla inglesa
df_reviews1['Review'] = df_reviews1['Review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

In [31]:
# Se buscan las filas vacias para luego eliminarlas
vacias = df_reviews1[~df_reviews1['Review'].str.contains(r'[a-zA-Z]')]
vacias

,Id_game,Review,Posted_date,Recommend
7,10,,2013-12-30,1
21,10,,2013-12-17,1
37,10,,2014-01-05,1
108,70,,2012-08-04,1
177,220,,2015-01-04,1
...,...,...,...,...
48961,413850,,2015-11-04,1
49055,417860,,2015-12-26,1
49057,417860,,2015-12-26,1
49171,423880,,2015-12-29,1


In [32]:
# Se eliminan las filas que estan vacias en la columna 'Review'
df_reviews1= df_reviews1[df_reviews1['Review'].str.contains(r'[a-zA-Z]')].reset_index()

Se pasa a eliminar las palabras stopworks del idioma ingles ya que estas son palabras comunes que no aportan mucha informacion para ello se descargaron los recursos necesarios paraesto como lo es ('wordnet') que contiene una base de datos lexica de lenguaje inglesa para la lematizacion (por ejemplo "jugando" se lematiza a  "jugar"), la desambiguación de palabras, búsqueda semántica y generación de lenguaje natural, tambien se descarga ('stopwords') el cual tiene una lista de palabras comunes de la lengua inglesa (ejemplo de stopwords: "a, an, of, for or, and, that...etc")

In [33]:
# Función para normalizar el texto y eliminar stopwords
def preprocess_text(text, language='english'):
    words = TextBlob(text).words
    words = [Word(word).lemmatize() for word in words if word.lower() not in stopwords.words(language)]
    return ' '.join(words)

In [34]:
# Se crea una nueva columna 'Normalized_review' donde contendra los valores procesados de la columna 'Review'
df_reviews1['Normalized_review'] = df_reviews1['Review'].apply(preprocess_text)

In [35]:
df_reviews1.head(7)

,index,Id_game,Review,Posted_date,Recommend,Normalized_review
0,0,10,gyertek gamelni mindenki itt van aki szmit d,2012-07-24,1,gyertek gamelni mindenki itt van aki szmit
1,1,10,good game d,2012-04-07,1,good game
2,2,10,go fk go,2014-03-23,1,go fk go
3,3,10,pure hardcore counterstrike in its rawest form...,2015-10-31,1,pure hardcore counterstrike rawest form godfat...
4,4,10,the best fps game,2014-01-22,1,best fps game
5,5,10,jueguenlooooooo,2013-08-16,1,jueguenlooooooo
6,6,10,lol,2015-11-07,1,lol


In [36]:
# Se crea la función de analisis de sentimiento
def sentiment_analysis(text):
    if isinstance(text, str) and len(text) > 0: # se valida que no este vacio el argumento y que sea una cadena de texto 
        analysis = TextBlob(text) # se crea una instancia de TextBlob
        sentiment_score = analysis.sentiment.polarity # 'sentiment_score' se usa para almacenar la puntuación de polaridad
        if sentiment_score < -0.2:
            return 0  # Malo
        elif sentiment_score > 0.2:
            return 2  # Positivo
        else:
            return 1  # Neutral
    else:
        return 1  # Valor por defecto si no hay reseña

In [37]:
# Se aplica la funcion anilisis de sentimiento a la columna 'Normalized_review'
df_reviews1['Sentiment_analysis']=df_reviews1['Normalized_review'].apply(sentiment_analysis)

In [42]:
# Se elimina las columnas que no se van a necesitar
df_reviews1 = df_reviews1.drop(columns=['Review','Normalized_review','index','Posted_date'])
df_reviews1.columns

Index(['Id_game', 'Recommend', 'Sentiment_analysis'], dtype='object')

In [41]:
df_reviews1

,Id_game,Posted_date,Recommend,Sentiment_analysis
0,10,2012-07-24,1,1
1,10,2012-04-07,1,1
2,10,2014-03-23,1,1
3,10,2015-10-31,1,2
4,10,2014-01-22,1,2
...,...,...,...,...
47979,423880,2015-12-04,1,1
47980,423880,2015-12-12,1,0
47981,423880,2015-12-23,1,0
47982,423880,2015-12-11,1,0


In [43]:
# Exportamos a csv
df_reviews1.to_csv('review_sentiment.csv', index=False)